<a target="_blank" rel="noopener noreferrer" href="https://colab.research.google.com/github/shawnrhoads/gu-psyc-347/blob/master/docs/module-01-02_Working-with-Data.ipynb">![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)</a>

# Working with Data

## Describing Data using Pandas
In this first section, we will use the Pandas package to explore and describe data from [O'Connell, K., Berluti, K., Rhoads, S. A., & Marsh, A. A. (2021). Reduced social distancing during the COVID-19 pandemic is associated with antisocial behaviors in an online United States sample. PLoS ONE.](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0244974)

This study assessed whether social distancing behaviors (early in the COVID-19 pandemic) was associated with self-reported antisocial behavior.

In [ ]:
# Remember: Python requires you to explictly "import" libraries before their functions are available to use. We will always specify our imports at the beginning of each notebook.
import pandas as pd, numpy as np
import requests # to download data from web

Here, we will load a dataset as a `pandas.DataFrame`, and investigate its attributes. We will see `N` rows for each subject, and `M` columns for each variable.

## Loading data

In [ ]:
# here, we are just going to download data from the web
url = 'https://raw.githubusercontent.com/shawnrhoads/gu-psyc-347/master/docs/static/data/OConnell_COVID_MTurk_noPII_post_peerreview.csv'
r = requests.get(url, allow_redirects=True)
filename = 'OConnell_COVID_MTurk_noPII_post_peerreview.csv' # string with location of file

open(filename, 'wb').write(r.content)

# load data specified in `filename` into dataframe `df`
df = pd.read_csv(filename)

In [ ]:
# check type of df
type(df)

In [ ]:
# how many rows and columns are in df?
print(df.values.shape) # N x M

Looks like we will have 131 rows (usually subjects, but can be multiple observations per subject) and 126 columns (usually variables)

In [ ]:
# let's output the first 5 rows of the df
print(df.head())

<hr>

### Creating custom DataFrames

We can also create our own dataframe. For example, here's a dataframe containing 20 rows and 3 columns of random numbers.

In [ ]:
sim_df = pd.DataFrame(np.random.randn(20, 3), index=range(0,20), columns=["column A", "column B", "column C"])
print(sim_df)

We can change the column names using list comprehension

In [ ]:
# e.g., change to upper case
sim_df.columns = [x.upper() for x in sim_df.columns]
print(sim_df.head()) #display first 5 rows

In [ ]:
# e.g., change to last element in string
sim_df.columns = [x[-1] for x in sim_df.columns]
print(sim_df.tail()) #display last 5 rows

<hr>

### Concatenating DataFrames

We can **concatenate** multiple dataframes containing the same columns (e.g., ['A','B','C']) using [pd.concat()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html). This will stack rows across dataframes.

Usage:
```
pd.concat(
    objs,
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)
```

In [ ]:
np.zeros((3,3))

In [ ]:
# create two new dataframes 

# the first will contain only zeros 
sim_df1 = pd.DataFrame(np.zeros((3, 3)), index=[20,21,22], columns=["A", "B", "C"])

# the second will contain only ones
sim_df2 = pd.DataFrame(np.ones((3, 3)), index=[23,24,25], columns=["A", "B", "C"])

In [ ]:
sim_dfs = [sim_df, sim_df1, sim_df2] # as list of dfs
result = pd.concat(sim_dfs)
print(result)

We can also concatenate using the rows (setting `axis=1`)

In [ ]:
# the first will contain only zeros 
sim_df3 = pd.DataFrame(np.zeros((3, 3)), index=[1,2,3], columns=["A", "B", "C"])

# the second will contain only ones
sim_df4 = pd.DataFrame(np.ones((3, 3)), index=[2,4,5], columns=["D", "E", "F"])

result2 = pd.concat([sim_df3, sim_df4], axis=1)
print(result2)

Notice that we have NaNs (not a number) in cells where there were no data (for example, no data in column `D` for index `1`)

We have to be careful because all elements will not merge across rows and columns by default. For example, if the second df also had a column "C", we will have two "C" columns by default.

In [ ]:
# the first will contain only zeros 
sim_df5 = pd.DataFrame(np.zeros((3, 3)), index=[1,2,3], columns=["A", "B", "C"])

# the second will contain only ones
sim_df6 = pd.DataFrame(np.ones((3, 4)), index=[2,4,5], columns=["C", "D", "E", "F"])

result3a = pd.concat([sim_df5, sim_df6], axis=1)
print(result3a)

By default, this method takes the "union" of dataframes. This is useful because it means no information will be lost!

But, now we have two columns named "C". To fix this, we can have pandas rename columns with matching names using the `DataFrame.merge()` method.

In [ ]:
result3b = sim_df5.merge(sim_df6, how='outer', left_index=True, right_index=True)
print(result3b)

We could also take the "intersection" across the two dataframes. We can do that by setting `join='inner'` (Meaning we only keep the rows that are shared between the two). In the previous case, this would be row '2'. All columns would be retained.

In [ ]:
result3c = pd.concat([sim_df5, sim_df6], axis=1, join='inner')
print(result3c)

<hr>

### Manipulating DataFrames

We can also change values within the dataframe using list comprehension.

In [ ]:
# first let's view the column "subID"
df['subID']

In [ ]:
# now, let's change these value by adding a prefix 'sub_' and store in a new column called "subID_2"
df['subID_2'] = ['sub_'+str(x) for x in df['subID']]

In [ ]:
df['subID_2']

We can also grab specific elements in the dataframe by specifying rows and columns

In [ ]:
print(df['age'][df['subID']==1001])

In [ ]:
# if you know the index (row name), then you can use the `pd.DataFrame.loc` method
df.loc[0,'age']

Creating new columns is particularly useful for computing new variables from old variables. For example: for each subject, let's multiply `age` by `STAB_total`.

In [ ]:
for index,subject in enumerate(df['subID']):
    df.loc[index,'new_col'] = df.loc[index,'age'] * df.loc[index,'STAB_total'] 

We can extract a column of observations to a numpy array

In [ ]:
sub_ids = df['subID'].values

In [ ]:
print(sub_ids)

In [ ]:
print(type(sub_ids))

We can also transpose the dataframe

In [ ]:
print(df.T.head())

### Statistics with DataFrames

We can compute all sorts of descriptive statistics on DataFrame columns using the following methods:
- `count()`: Number of non-null observations
- `sum()`: Sum of values
- `mean()`: Mean of values
- `median()`: Median of values
- `mode()`: Mode of values
- `std()`: Standard deviation of values
- `min()`: Minimum value
- `max()`: Maximum value
- `abs()`: Absolute value
- `prod()`: Product of values
- `cumsum()`: Cumulative sum
- `cumprod()`: Cumulative product

Here are some examples:

In [ ]:
# mean of a column
df["age"].mean()

In [ ]:
# mean of multiple columns
df[["age","STAB_total"]].mean()

In [ ]:
# median of a column
df["age"].median()

In [ ]:
# compute a summary of metrics on columns
df[["age", "STAB_total"]].describe()

In [ ]:
# group means by sex
df.groupby("sex")[["age", "STAB_total"]].mean()

In [ ]:
# group means by sex and education
df.groupby(["sex","education_coded"])[["age", "STAB_total"]].mean()

In [ ]:
# group counts by sex and education
df.groupby(["sex","education_coded"])[["age", "STAB_total"]].count()

We can also correlate 2 or more variables

In [ ]:
df[["age","STAB_total","socialdistancing"]].corr(method="spearman")

## Visualizing Data using Matplotlib

To understand what our data look like, we will visualize it in different ways.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Let's plot the distribiton of one variable in our data

In [ ]:
plt.hist(df['age'], bins=9)
plt.xlabel("Age")
plt.ylabel("Number of Subjects")

Let's see what percentage of subjects have a below-average score:

In [ ]:
mean_age = np.mean(df['age'])
frac_below_mean = (df['age'] < mean_age).mean()
print(f"{frac_below_mean:2.1%} of subjects are below the mean")

We can also see this by adding the average score to the histogram plot:


In [ ]:
plt.hist(df['age'], bins=9)
plt.xlabel("Age")
plt.ylabel("Number of Subjects")

plt.axvline(mean_age, color="orange", label="Mean Age")
plt.legend()

Comparing mean and median

In [ ]:
med_age = np.median(df['age'])

plt.hist(df['age'], bins=9)
plt.xlabel("Age")
plt.ylabel("Number of Subjects")

plt.axvline(mean_age, color="orange", label="Mean Age")
plt.axvline(med_age, color="black", label="Median Age")

plt.legend()